In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def logic_win(x,x_r):
    return (x >= x_r[0]) & (x < x_r[1])

#### Import and clean dataset, select out London

In [2]:
path = 'combined_collisions_v3.csv'

df = pd.read_csv(path,low_memory=False)
df.drop(columns='Unnamed: 0',inplace=True)

In [3]:
#Time column has some nans in it, must drop those entries

def is_str(x):
    Nx = len(x)
    I = np.ones(Nx).astype(bool)
    for ii in range(Nx):
        if type(x[ii]) != str:
            I[ii] = False
    return I

df = df.loc[is_str(df.Time.values),:]
df.aadf_Count_point_id = df.aadf_Count_point_id.values.astype(int)

In [4]:
#make hour, year and month columns
    #hour will be rounded 
def get_hour(T):
    Nt = len(T)
    H = np.zeros(Nt).astype(int)
    for ii in range(Nt):
        s = T[ii].split(':')
        h = int(s[0])
        m = int(s[1])
        if m > 30:
            h += 1
        H[ii] = h % 24
    return H

def get_ymd(D):
    Nd = len(D)
    # |
    ymd = np.zeros((Nd,3)).astype(int)
    for ii in range(Nd):
        s = D[ii].split('-')
        for nn in range(3):
            ymd[ii,nn] = int(s[nn])
    return ymd

def reformat_aadf_year(Year):
    Ny = len(Year)
    y = np.zeros(Ny).astype(int)
    for ii in range(Ny):
        y[ii] = int(Year[ii].split('-')[0])
    return y

df[['y','m','d']] = get_ymd(df.Date.values)
df['h'] = get_hour(df.Time.values)
#df['aadf_Year'] = reformat_aadf_year(df.aadf_Year.values)

##### Below selects out region. You can use Dan's feature to select Greater London, or code below for central London.
- additionally I drop all entries with no traffic counter matches

In [5]:
#central London box
Lat_r = [51.475,51.543]
Lon_r = [-0.179,0.023]

I_lat = (df.Latitude > Lat_r[0]) & (df.Latitude < Lat_r[1])
I_lon = (df.Longitude > Lon_r[0]) & (df.Longitude < Lon_r[1])
I = I_lat & I_lon

#df_lond = df.loc[I,:].copy() #selects central London
df_lond = df.loc[df.in_london,:].copy() #this selects Greater London

In [6]:
#drop all data that don't have traffic matches

I = df_lond.match == True
df_lond = df_lond.loc[I,:]
df_lond.aadf_Year = reformat_aadf_year(df_lond.aadf_Year.values)

#### Make neighborhoods feature

In [7]:
#Make a 3 (Lat, N/S) x 6 (Lon, E/W) grid
Nlat = 3
Nlon = 6
Nneigh = Nlon*Nlat

edges_lat = np.linspace(*Lat_r,Nlat+1)
dlat = np.diff(edges_lat)[0]
bins_lat = edges_lat[:-1]+0.5*dlat
edges_lon = np.linspace(*Lon_r,Nlon+1)
dlon = np.diff(edges_lon)[0]
bins_lon = edges_lon[:-1]+0.5*dlon

coord_neigh = [None]*Nneigh

df_lond['neigh'] = 0 #initialize neighborhood feature
ir = 0 #these are lat bins (rows)
ic = 0 #lon bins
for ii in range(Nneigh):
    Ilat = logic_win(df_lond.Latitude,(edges_lat[ir],edges_lat[ir+1]))
    Ilon = logic_win(df_lond.Longitude,(edges_lon[ic],edges_lon[ic+1]))
    I = Ilat & Ilon
    df_lond.loc[I,'neigh'] = ii
    
    coord_neigh[ii] = np.array([bins_lat[ir],bins_lon[ic]])
    
    ic+=1
    if ic == Nlon:
        ic = 0
        ir+=1

In [8]:
#Make a feature for if there is a bike lane of not ('bikelane'). There is no bike lane if all
#the below classifiers are False

f_laneClass = ['aadf_CLT_CARR', 'aadf_CLT_SEGREG', 'aadf_CLT_STEPP', 'aadf_CLT_PARSEG',
       'aadf_CLT_SHARED', 'aadf_CLT_MANDAT', 'aadf_CLT_ADVIS',
       'aadf_CLT_PRIORI', 'aadf_CLT_CONTRA', 'aadf_CLT_BIDIRE',
       'aadf_CLT_CBYPAS', 'aadf_CLT_BBYPAS', 'aadf_CLT_PARKR',
       'aadf_CLT_WATERR', 'aadf_CLT_PTIME']

Inolane = np.ones(df_lond.shape[0]).astype(bool)
for f in f_laneClass:
    Inolane &= (df_lond[f].values == False)
    
df_lond['bikelane'] = ~Inolane

In [9]:
traf_IDs = np.array(df_lond.aadf_Count_point_id.value_counts().index)
Ninc_IDs = np.array(df_lond.aadf_Count_point_id.value_counts().values)

######################################
Nthresh = 1 #this sets which traffic counters are included, they must have at least this many accidents 
######################################

I = Ninc_IDs >= Nthresh #this will select roads to include in the new DataFrame based on if they had at least this many accidents
I.sum()

IDs_df = traf_IDs[I]
Nid = len(IDs_df)

df_road = pd.DataFrame({'ID':IDs_df})
slight_rd = np.zeros(Nid).astype(int)
ser_rd = np.zeros(Nid).astype(int)
total_rd = np.zeros(Nid).astype(int)
for nn,ii in enumerate(IDs_df):
    I = df_lond.aadf_Count_point_id.values == ii
    total_rd[nn] = I.sum()
    slight_rd[nn] = df_lond.loc[I,'slight'].values.sum()
    ser_rd[nn] = total_rd[nn] - slight_rd[nn]
df_road['total'] = total_rd
df_road['slight'] = slight_rd
df_road['serious'] = ser_rd

safety_mode = ['neigh','Road_name','Road_name2','h','Day_of_Week','m','Speed_limit','Junction_Detail','Junction_Control','Pedestrian_Crossing-Physical_Facilities']
aadf_av = ['distance_to_cp','aadf_Pedal_cycles','aadf_Pedal_cycles','aadf_All_motor_vehicles','aadf_All_HGVs','aadf_LGVs','aadf_Buses_and_coaches']
aadf_mode = ['bikelane','aadf_Road_name','aadf_Road_category','aadf_Road_type','aadf_CLT_CARR', 'aadf_CLT_SEGREG', 'aadf_CLT_STEPP', 'aadf_CLT_PARSEG',
       'aadf_CLT_SHARED', 'aadf_CLT_MANDAT', 'aadf_CLT_ADVIS',
       'aadf_CLT_PRIORI', 'aadf_CLT_CONTRA', 'aadf_CLT_BIDIRE',
       'aadf_CLT_CBYPAS', 'aadf_CLT_BBYPAS', 'aadf_CLT_PARKR',
       'aadf_CLT_WATERR', 'aadf_CLT_PTIME','aadf_Link_length_miles']

for f in safety_mode:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mode().values[0]

for f in aadf_mode:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mode().values[0]
        
for f in aadf_av:
    df_road[f] = np.zeros(Nid).astype(int)
    for nn,ii in enumerate(IDs_df):
        I = df_lond.aadf_Count_point_id.values == ii
        df_road.loc[nn,f] = df_lond.loc[I,f].mean()
        
median_d = df_road.distance_to_cp.median()
median_bv = df_road.aadf_Pedal_cycles.median()
df_road['total_dnorm'] = df_road['total'].values*median_d/df_road['distance_to_cp'].values
df_road['total_dnorm_bvnorm'] = df_road.total_dnorm.values*median_bv/df_road.aadf_Pedal_cycles.values
df_road['serious_ratio'] = (df_road['serious'].values)/df_road['total'].values

C:\Users\mccracken.96\AppData\Local\Temp\1\ipykernel_13600\4276915281.py:56: RuntimeWarning: divide by zero encountered in divide
  df_road['total_dnorm_bvnorm'] = df_road.total_dnorm.values*median_bv/df_road.aadf_Pedal_cycles.values


##### Merge in roads that had no accidents

In [10]:
df_allroad = pd.read_csv('aadf_features_london_colyears.csv')

In [11]:
all_IDs = df_allroad.Count_point_id.value_counts().index

#find the IDs that are missing in above df_road, these are the ones with no accidents
Izero = ~np.isin(all_IDs,traf_IDs)
zero_IDs = all_IDs[Izero]

In [12]:
#now go through and make a df formatted exactly the same as df_road for the zero accident roads

road_cols = np.array(df_road.columns)
#get features that are already in df_allroads
allroad_cols = df_allroad.columns
allroad_colsrename = {}
for ii,f in enumerate(allroad_cols):
    allroad_colsrename[f] = 'aadf_'+f
    
df_allroad.rename(allroad_colsrename,inplace=True,axis='columns')
df_allroad.rename({'aadf_Count_point_id':'ID'},inplace=True,axis='columns')

In [13]:
df_road_zero = pd.DataFrame()
df_road_zero['ID'] = zero_IDs
Nzero = Izero.sum()
Inotin = ~np.isin(road_cols,df_allroad.columns)
fzeros = np.array(['total','slight','serious','total_dnorm','total_dnorm_bvnorm'])
for f in road_cols[Inotin]:
    if np.isin(f,fzeros):
        df_road_zero[f] = np.zeros(Nzero)
    else:
        df_road_zero[f] = -9*np.ones(Nzero)
    

f_mode = ['aadf_Road_name','aadf_Road_category','aadf_Road_type','aadf_CLT_CARR', 'aadf_CLT_SEGREG', 'aadf_CLT_STEPP', 'aadf_CLT_PARSEG',
       'aadf_CLT_SHARED', 'aadf_CLT_MANDAT', 'aadf_CLT_ADVIS',
       'aadf_CLT_PRIORI', 'aadf_CLT_CONTRA', 'aadf_CLT_BIDIRE',
       'aadf_CLT_CBYPAS', 'aadf_CLT_BBYPAS', 'aadf_CLT_PARKR',
       'aadf_CLT_WATERR', 'aadf_CLT_PTIME'] 
f_av = ['aadf_Link_length_miles','aadf_Pedal_cycles','aadf_All_motor_vehicles','aadf_All_HGVs','aadf_LGVs','aadf_Buses_and_coaches']
for f in f_mode:
    A = [None]*Nzero
    for ii,rid in enumerate(zero_IDs):
        I = df_allroad.ID == rid
        A[ii] = df_allroad.loc[I,f].mode().values[0]
    df_road_zero[f] = A
    
for f in f_av:
    A = [None]*Nzero
    for ii,rid in enumerate(zero_IDs):
        I = df_allroad.ID == rid
        A[ii] = df_allroad.loc[I,f].mean()
    df_road_zero[f] = A
    
Inan = np.isnan(df_road_zero.aadf_Link_length_miles.values)
df_road_zero.loc[Inan,'aadf_Link_length_miles'] = -9

In [14]:
f_inttype = ['total', 'slight', 'serious', 'neigh', 'Road_name', 'Road_name2',
       'h', 'Day_of_Week', 'm', 'Speed_limit', 'Junction_Detail',
       'Junction_Control', 'Pedestrian_Crossing-Physical_Facilities']
f_str_accident = ['Road_name','Road_name2']

for f in f_inttype:
    df_road_zero[f] = df_road_zero[f].values.astype(int)
    
for f in f_str_accident:
    df_road_zero[f] = 'N/A'

In [15]:
#lastly need to set 'bikelane feature'

f_laneClass = ['aadf_CLT_CARR', 'aadf_CLT_SEGREG', 'aadf_CLT_STEPP', 'aadf_CLT_PARSEG',
       'aadf_CLT_SHARED', 'aadf_CLT_MANDAT', 'aadf_CLT_ADVIS',
       'aadf_CLT_PRIORI', 'aadf_CLT_CONTRA', 'aadf_CLT_BIDIRE',
       'aadf_CLT_CBYPAS', 'aadf_CLT_BBYPAS', 'aadf_CLT_PARKR',
       'aadf_CLT_WATERR', 'aadf_CLT_PTIME']

Inolane = np.ones(df_road_zero.shape[0]).astype(bool)
for f in f_laneClass:
    Inolane &= (df_road_zero[f].values == False)
    
df_road_zero['bikelane'] = ~Inolane

##### Now we can concatenate df_road_zero into df_road

In [16]:
df_road = pd.concat([df_road,df_road_zero],ignore_index=True)

##### Add metadata for survey years

In [17]:
#add in aadf year availability metadata 
path_year = 'aadf_year_meta.csv'
df_year = pd.read_csv(path_year)

Nroadall = df_road.shape[0]

#merge the metadata into df_road
feat_add = list(df_year.columns)
feat_add.remove('Count_point_id')
for c in feat_add:
    df_road[c] = np.zeros(Nroadall).astype(int)
    for ii,ID in enumerate(df_road.ID.values):
        I = df_year.Count_point_id == ID
        df_road.loc[ii,c] = df_year.loc[I,c].values

#### Here is the final DataFrame, feel free to save your own version

In [18]:
df_road.sample(5)

,ID,total,slight,serious,neigh,Road_name,Road_name2,h,Day_of_Week,m,...,aadf_All_HGVs,aadf_LGVs,aadf_Buses_and_coaches,total_dnorm,total_dnorm_bvnorm,serious_ratio,N_year,Year_start,Year_end,Max_year_gap
1116,6785,8,8,0,0,A224,U,11,3,5,...,428.500000,2393.875000,124.625000,4.239398,24.771906,0.000000,22,2000,2021,1
902,26797,11,10,1,0,A216,U,9,3,4,...,490.818182,2151.363636,384.909091,7.185800,5.089031,0.090909,22,2000,2021,1
2257,17019,0,0,0,-9,N/A,N/A,-9,-9,-9,...,7314.545455,15072.000000,530.090909,0.000000,0.000000,-9.000000,16,2000,2015,1
2330,942337,0,0,0,-9,N/A,N/A,-9,-9,-9,...,105.200000,607.900000,507.600000,0.000000,0.000000,-9.000000,11,2009,2019,1
984,27237,9,8,1,0,A502,U,9,2,5,...,172.000000,1766.555556,414.222222,3.948038,9.302286,0.111111,22,2000,2021,1


In [18]:
#save dataframe

path = 'df_road_v3_zeros.csv'
df_road.to_csv(path,index_label=False)